## Excursus: Creating Technical Indicators with Python and Pandas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Loading the Dataset

In [ ]:
data = pd.read_csv("fin_data.csv", index_col = "Datetime", usecols = ["Datetime", "Open", "High", "Low", "Close"])
data

In [ ]:
data.index = pd.to_datetime(data.index)

In [ ]:
data.info()

### Simple Moving Averages (SMA ratio)

In [ ]:
data.Close.plot()
plt.ylabel("EUR/USD")
plt.title("EUR/USD FX Rate")
plt.show()

In [ ]:
sma_s = 50
sma_l = 200

In [ ]:
data["SMA_S"] = data.Close.rolling(sma_s).mean()

In [ ]:
data["SMA_L"] = data.Close.rolling(sma_l).mean()

In [ ]:
data

In [ ]:
data.loc["2023-07", ["Close", "SMA_S", "SMA_L"]].plot(figsize = (12, 8), title = "EUR/USD - SMA{} | SMA{}".format(sma_s, sma_l), fontsize = 12)
plt.legend(fontsize = 12)
plt.show()

In [ ]:
data["SMA_ratio"] = data.SMA_L / data.SMA_S
data

In [ ]:
data.loc[:, ["SMA_ratio"]].plot(figsize = (12, 8), title = "EUR/USD - SMA{} | SMA{}".format(sma_s, sma_l), fontsize = 12)
plt.legend(fontsize = 12)
plt.show()

__SMA Definition:__

SMA (Simple Moving Average): It's the average of a specified number of the most recent closing prices. For instance, a 50-period SMA is the average of the last 50 closing prices.

__Parameters:__<br>
sma_s = 50: This sets the window size for the short-term SMA to 50 periods (hours, in this case).<br>
sma_l = 200: This sets the window size for the long-term SMA to 200 periods (hours, in this case).<br>

__Calculating the Short-Term SMA:__<br>
data["SMA_S"] = data.Close.rolling(sma_s).mean():<br>
This line calculates the 50-period SMA for the Close price.<br>
The .rolling(sma_s) method creates a rolling window of 50 periods.<br>
The .mean() function computes the average within this window.<br>
The result is stored in a new column SMA_S.<br>

__Calculating the Long-Term SMA:__<br>
data["SMA_L"] = data.Close.rolling(sma_l).mean():<br>
Similarly, this line calculates the 200-period SMA for the Close price.<br>
It uses a rolling window of 200 periods to compute the average.<br>
The result is stored in another new column SMA_L.<br>

__Calculating the SMA Ratio:__<br>
data["SMA_ratio"] = data.SMA_L / data.SMA_S:<br>
This line calculates the ratio between the long-term SMA (SMA_L) and the short-term SMA (SMA_S).<br>
This ratio indicates the relative position of the short-term average compared to the long-term average.<br>
The resulting ratio is stored in a new column SMA_ratio.<br>

__Purpose of the SMA Ratio:__<br>
The SMA ratio can help identify trends in the market:<br>
If SMA_ratio > 1: The long-term average is higher than the short-term average, possibly indicating a downward trend.<br>
If SMA_ratio < 1: The long-term average is lower than the short-term average, possibly indicating an upward trend.<br>
By using these calculations, you can analyze the momentum and trend direction of the EUR/USD currency pair over time.

### MACD Histogram

In [ ]:
ema_s = 12 # EMA Short
ema_l = 26 # EMA Long

In [ ]:
data["EMA_S"] = data.Close.ewm(span = ema_s, min_periods = ema_s).mean() 

In [ ]:
data["EMA_L"] = data.Close.ewm(span = ema_l, min_periods = ema_l).mean() 

In [ ]:
data

In [ ]:
data.loc["2023-07", ["Close", "EMA_S", "EMA_L"]].plot(figsize = (12, 8))
plt.show()

In [ ]:
data["MACD"] = data.EMA_S - data.EMA_L
data

In [ ]:
signal_mw = 9 # Moving Window for Signal Line

In [ ]:
data["MACD_Signal"] = data.MACD.ewm(span = signal_mw, min_periods = signal_mw).mean() 
data

In [ ]:
data["MACD_hist"] = data.MACD - data.MACD_Signal
data

In [ ]:
data.loc["2023", ["MACD_hist"]].plot(figsize = (12, 8), fontsize = 12)
plt.legend(fontsize = 12)
plt.show()

1. MACD (Moving Average Convergence Divergence):
MACD is a trend-following momentum indicator that shows the relationship between two moving averages of a security’s price.<br>
Histogram: The difference between the MACD line and the MACD Signal line. It visually represents the strength and direction of the trend.
2. Parameters:
ema_s = 12: This sets the period for the short-term Exponential Moving Average (EMA) to 12 periods (hours in this context).<br>
ema_l = 26: This sets the period for the long-term EMA to 26 periods (hours in this context).
3. Calculating the Short-Term EMA:
data["EMA_S"] = data.Close.ewm(span = ema_s, min_periods = ema_s).mean():<br>
This calculates the 12-period EMA of the Close price.<br>
.ewm(span = ema_s) applies an exponential weighting, giving more importance to recent data.<br>
The .mean() function computes the average, which updates as new data points are added.
The result is stored in a new column EMA_S.
4. Calculating the Long-Term EMA:
data["EMA_L"] = data.Close.ewm(span = ema_l, min_periods = ema_l).mean():<br>
Similarly, this calculates the 26-period EMA of the Close price.<br>
It uses a longer window, giving a smoother average that reacts more slowly to price changes.
The result is stored in a new column EMA_L.
5. Plotting the Close Price and EMAs:
data.loc["2023-07", ["Close", "EMA_S", "EMA_L"]].plot(figsize = (12, 8)):<br>
This line plots the Close price, Short-term EMA (12), and Long-term EMA (26) for July 2023.<br>
loc["2023-07"] filters the data for the specified month.<br>
The plot helps visualize how the short and long EMAs compare to the Close price.
6. Calculating the MACD Line:
data["MACD"] = data.EMA_S - data.EMA_L:<br>
This line calculates the difference between the short-term EMA (EMA_S) and the long-term EMA (EMA_L).<br>
The result is the MACD line, which oscillates above and below zero, indicating momentum direction.<br>
The MACD line is stored in a new column MACD.
7. Calculating the Signal Line:
signal_mw = 9: This sets the period for the Signal line to 9 periods (hours).<br>
data["MACD_Signal"] = data.MACD.ewm(span = signal_mw, min_periods = signal_mw).mean():<br>
The Signal line is a 9-period EMA of the MACD line.<br>
This line smooths the MACD to make it easier to interpret trend changes.<br>
The result is stored in a new column MACD_Signal.<br>
8. Calculating the MACD Histogram:
data["MACD_hist"] = data.MACD - data.MACD_Signal:<br>
The MACD Histogram is the difference between the MACD line and the Signal line.<br>
It visually represents the momentum of the trend, with positive values indicating upward momentum and negative values indicating downward momentum.<br>
The result is stored in a new column MACD_hist.
9. Purpose of the MACD Histogram:
Positive Histogram: When the MACD is above the Signal line, indicating bullish momentum.<br>
Negative Histogram: When the MACD is below the Signal line, indicating bearish momentum.<br>
The histogram bars help traders identify potential buy/sell signals based on the strength of the trend.<br>
By using these calculations, you can analyze the momentum and possible trend reversals in the EUR/USD currency pair.

### RSI

In [ ]:
data["U"] = np.where(data.Close.diff() > 0, data.Close.diff(), 0)

In [ ]:
data["D"] = np.where(data.Close.diff() < 0, -data.Close.diff(), 0)

In [ ]:
data

In [ ]:
periods = 20

In [ ]:
data["MA_U"] = data.U.rolling(periods).mean()

In [ ]:
data["MA_D"] = data.D.rolling(periods).mean()

In [ ]:
data["RSI"] = data.MA_U / (data.MA_U + data.MA_D) * 100

In [ ]:
data.loc["2023-10", ["Close", "RSI"]].plot(figsize = (12, 8), secondary_y = "RSI")
plt.show()

1. RSI (Relative Strength Index):
RSI is a momentum oscillator that measures the speed and change of price movements. It ranges from 0 to 100 and is used to identify overbought or oversold conditions in the market.
2. Calculating Upward and Downward Price Movements:
data["U"] = np.where(data.Close.diff() > 0, data.Close.diff(), 0):<br>
This line calculates the difference between the current and previous close prices using data.Close.diff().<br>
If the difference is positive (indicating a price increase), the value is recorded in the U column.<br>
If the difference is not positive, the value in the U column is set to 0 (indicating no upward movement).<br>
data["D"] = np.where(data.Close.diff() < 0, -data.Close.diff(), 0):<br>
Similarly, this calculates the downward price movements.<br>
If the difference is negative (indicating a price decrease), the absolute value is recorded in the D column.<br>
If the difference is not negative, the value in the D column is set to 0 (indicating no downward movement).<br>
3. Setting the RSI Period:
periods = 20:<br>
This sets the period over which the RSI will be calculated, typically 14 or 20 periods are used.<br>
4. Calculating the Moving Averages of Upward and Downward Movements:
data["MA_U"] = data.U.rolling(periods).mean():<br>
This line calculates the moving average of the upward price movements over the specified period.<br>
It creates a rolling window of 20 periods and computes the average of the U column.<br>
The result is stored in a new column MA_U.<br>
data["MA_D"] = data.D.rolling(periods).mean():<br>
Similarly, this calculates the moving average of the downward price movements over the specified period.<br>
It creates a rolling window of 20 periods and computes the average of the D column.<br>
The result is stored in a new column MA_D.
5. Calculating the RSI:
data["RSI"] = data.MA_U / (data.MA_U + data.MA_D) * 100:<br>
The RSI is calculated by dividing the moving average of upward movements (MA_U) by the sum of the moving averages of upward (MA_U) and downward (MA_D) movements.<br>
This ratio is then multiplied by 100 to convert it to a percentage.<br>
The resulting RSI value ranges from 0 to 100, indicating the strength of recent price movements.<br>
The result is stored in a new column RSI.<br>
6. Purpose of the RSI:
Overbought Condition: Typically, an RSI above 70 indicates that the asset may be overbought and could be due for a correction or pullback.<br>
Oversold Condition: An RSI below 30 suggests that the asset may be oversold and could be due for a rebound.<br>
Trend Indicator: RSI values between 30 and 70 can also indicate the strength of the current trend, with values closer to 50 suggesting a more neutral momentum.<br>

By using this RSI calculation, traders can analyze whether the EUR/USD currency pair is potentially overbought or oversold, helping them make informed trading decisions.

### Stochastic Oscillator (difference %K minus %D)

In [ ]:
periods = 14

In [ ]:
data["roll_low"] = data.Low.rolling(periods).min()
data["roll_high"] = data.High.rolling(periods).max()

In [ ]:
data

In [ ]:
data.loc["2023-10", ["Close", "roll_low", "roll_high"]].plot(figsize = (12, 8), fontsize = 12)
plt.legend(fontsize = 12)
plt.show()

In [ ]:
data["K"] = (data.Close - data.roll_low) / (data.roll_high - data.roll_low) * 100

In [ ]:
moving_av = 3

In [ ]:
data["D"] = data.K.rolling(moving_av).mean()

In [ ]:
data

In [ ]:
data.loc["2023-10-17", ["D", "K"]].plot(figsize = (12, 8), fontsize = 12)
plt.show()

In [ ]:
(data["K"] - data["D"]).plot()

In [ ]:
data["SO_diff"] = data["K"] - data["D"]
data

In [ ]:
data.loc["2023-10", "SO_diff"].plot(figsize = (12, 8), fontsize = 12)
plt.show()

1. Stochastic Oscillator Overview:
The Stochastic Oscillator is a momentum indicator that compares the closing price of an asset to its price range over a specified period. It is used to identify potential overbought or oversold conditions in the market.
2. Setting the Period for Stochastic Calculation:
periods = 14:<br>
This sets the look-back period to 14, which is a common setting for calculating the Stochastic Oscillator.
It will consider the last 14 periods (hours, in this context) for the calculation.
3. Calculating the Rolling Low and High:
data["roll_low"] = data.Low.rolling(periods).min():<br>
This line calculates the lowest low over the last 14 periods.<br>
It creates a rolling window of 14 periods and finds the minimum value in the Low column.<br>
The result is stored in a new column roll_low.<br>
data["roll_high"] = data.High.rolling(periods).max():<br>
Similarly, this calculates the highest high over the last 14 periods.<br>
It creates a rolling window of 14 periods and finds the maximum value in the High column.<br>
The result is stored in a new column roll_high.
4. Calculating the %K Line (Fast Stochastic Oscillator):
data["K"] = (data.Close - data.roll_low) / (data.roll_high - data.roll_low) * 100:<br>
The %K line represents the current close price relative to the 14-period price range.<br>
It is calculated by subtracting the 14-period low (roll_low) from the current close price and dividing the result by the difference between the 14-period high (roll_high) and the 14-period low (roll_low).<br>
This ratio is then multiplied by 100 to convert it into a percentage, indicating the position of the close price within the 14-period range.<br>
The result is stored in a new column K.
5. Setting the Moving Average Period for %D Line:
moving_av = 3:<br>
This sets the period for calculating the %D line, which is a 3-period moving average of the %K line.
6. Calculating the %D Line (Slow Stochastic Oscillator):
data["D"] = data.K.rolling(moving_av).mean():<br>
The %D line is a 3-period simple moving average of the %K line.<br>
It smooths out the %K values to provide a clearer signal.<br>
The result is stored in a new column D.
7. Calculating the Stochastic Oscillator Difference:
data["SO_diff"] = data["K"] - data["D"]:<br>
This line calculates the difference between the %K line and the %D line.<br>
The result is stored in a new column SO_diff.<br>
This difference can indicate potential buy/sell signals:<br>
Positive SO_diff: When %K is above %D, it suggests upward momentum.<br>
Negative SO_diff: When %K is below %D, it suggests downward momentum.
8. Purpose of the Stochastic Oscillator:
Overbought Condition: Typically, values above 80 suggest that the asset may be overbought and could be due for a pullback.<br>
Oversold Condition: Values below 20 suggest that the asset may be oversold and could be due for a rebound.<br>
Trend Indicator: Crossovers of the %K and %D lines can indicate potential trend reversals or continuation.<br>
By using these calculations, you can analyze the momentum of the EUR/USD currency pair over time and identify potential turning points in the market.